In [3]:
import csv
# !pip install gtts
from gtts import gTTS
import os

# Accessibility settings
HIGH_CONTRAST = "\033[1;37;44m"  # Bold White on Blue
LARGE_TEXT = "\033[1;37m"        # Bold White Text
RESET = "\033[0m"

# Speak text using gTTS
def speak_text(text):
    try:
        tts = gTTS(text, lang='en')
        tts.save("audio.mp3")
        # Use mpg321 or afplay depending on your system
        if os.name == 'posix':
            os.system("mpg321 output.mp3 2>/dev/null")  # Linux
        else:
            os.system("start output.mp3")  # Windows
    except Exception as e:
        print(f"[Speech error: {e}]")

# Beep sound for confirmation
def beep():
    try:
        os.system('echo -e "\a"')  # Works on Unix systems
    except:
        print("[Beep]")

# Removed speech_to_text function and mic input
# Ask user for input only via typing
def get_input(prompt):
    return input(prompt)

# Load resources from CSV file
def load_resources(filename):
    resources = []
    try:
        with open(filename, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                row['Services'] = [s.strip().lower() for s in row['Services'].split(';')]
                row['WheelchairAccessible'] = row['WheelchairAccessible'].strip().lower() == "yes"
                resources.append(row)
        return resources
    except Exception as e:
        print(f"Error loading resources: {e}")
        return []

# Display resources with accessibility options
def print_resources(resources, font_size=1, highlight=HIGH_CONTRAST):
    if not resources:
        msg = highlight + "No resources found." + RESET
        print(msg)
        speak_text("No resources found.")
        return

    for i, res in enumerate(resources, start=1):
        accessible = "Yes" if res["WheelchairAccessible"] else "No"
        indent = ' ' * (font_size * 2)
        info = (
            f"{highlight}{i}. {res['Name']}{RESET}\n"
            f"{indent}Address: {res['Address']}\n"
            f"{indent}City: {res['City']}, State: {res['State']}, ZIP: {res['ZIP']}\n"
            f"{indent}Phone: {res['Phone']}\n"
            f"{indent}Wheelchair Accessible: {accessible}\n"
            f"{indent}Services: {', '.join(res['Services'])}\n"
        )
        print(info)
        speak_text(f"Resource {i}: {res['Name']}, located at {res['Address']}, {res['City']}, {res['State']}. Phone: {res['Phone']}. Wheelchair accessible: {accessible}. Services include {', '.join(res['Services'])}.")

# Store reminders in memory
reminders = []

def add_reminder():
    print("\n--- Add a Health Appointment Reminder ---")
    date_time = get_input("Enter date and time (YYYY-MM-DD HH:MM): ").strip()
    description = get_input("Enter reminder description: ").strip()
    location = get_input("Enter location: ").strip()
    phone = get_input("Enter phone number: ").strip()

    reminders.append({
        "date_time": date_time,
        "description": description,
        "location": location,
        "phone": phone
    })
    print("Reminder added!")
    speak_text("Reminder added!")

def list_reminders():
    if not reminders:
        print("No reminders set.")
        speak_text("You have no reminders set.")
        return
    print("\n--- Your Health Appointment Reminders ---")
    for i, rem in enumerate(reminders, 1):
        reminder_text = f"{i}. {rem['date_time']} - {rem['description']} at {rem['location']} (Phone: {rem['phone']})"
        print(reminder_text)
        speak_text(reminder_text)
    print()

# Main program
def main():
    print(f"{HIGH_CONTRAST}Welcome to the Accessible Health Resource Finder and Reminder!{RESET}")
    speak_text("Welcome to the Accessible Health Resource Finder and Reminder!")
    beep()

    # Accessibility settings
    has_disability = get_input("Do you have a disability? (yes/no): ").strip().lower()
    font_size = 1
    highlight = HIGH_CONTRAST if has_disability == "yes" else RESET

    if has_disability == "yes":
        try:
            font_size = int(get_input("Choose font size (1 for normal, 2 for large): ").strip())
            if font_size not in [1, 2]:
                font_size = 1
        except ValueError:
            font_size = 1

    # Load resources
    resources = load_resources("resources.csv")
    if not resources:
        print("No resource data available. Please ensure 'resources.csv' is present.")
        speak_text("No resource data available. Please ensure the resources CSV file is present.")
        return

    # Get user inputs (only typed)
    city = get_input("Enter city: ").strip().lower()
    zip_code = get_input("Enter ZIP code: ").strip()
    state = get_input("Enter state: ").strip().lower()
    service = get_input("What service are you looking for? ").strip().lower()

    # Filter resources
    filtered = [res for res in resources if
                (city in res["City"].lower() or not city) and
                (zip_code == res["ZIP"] or not zip_code) and
                (state in res["State"].lower() or not state) and
                (any(service in s for s in res['Services']) or not service)
               ]

    print_resources(filtered, font_size, highlight)

    # Reminder loop
    while True:
        add_rem = get_input("Would you like to add a health appointment reminder? (yes/no): ").strip().lower()
        if add_rem == 'yes':
            add_reminder()
        else:
            break

    list_reminders()
    print("Thank you for using the Accessible Health Resource Finder and Reminder!")
    speak_text("Thank you for using the Accessible Health Resource Finder and Reminder!")

if __name__ == "__main__":
    main()


Welcome to the Accessible Health Resource Finder and Reminder!
Do you have a disability? (yes/no): yes
Choose font size (1 for normal, 2 for large): 1
Enter city: durham
Enter ZIP code: 27751
Enter state: nc
What service are you looking for? counseling
No resources found.
Would you like to add a health appointment reminder? (yes/no): no
No reminders set.
Thank you for using the Accessible Health Resource Finder and Reminder!
